# 0. Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dacon_RF_parameter_tuning

/content/drive/MyDrive/Dacon_RF_parameter_tuning


In [ ]:
ls

 0328_submit.ipynb            github_upload/          submit0330_2.csv          train.csv
 0330_Optuna.ipynb            grid_test2.ipynb        submit.ipynb
 BayesianOptimization.ipynb   sample_submission.csv   test_grid.ipynb
 EDA.ipynb                    submit/                'test_grid.ipynb의 사본'


In [ ]:
# 대회 제공 시드로 고정
import random
random.seed(42)

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1


In [ ]:
X = train.drop(labels='login',axis=1)
X
y = train['login']
y

0       1
1       1
2       1
3       1
4       1
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: login, Length: 1309, dtype: int64

# 1. Optuna

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
# valid data와 test data를 모두 test data로 사용한 경우
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    model = RandomForestClassifier(**params)

    # 훈련 데이터를 훈련 세트와 검증 세트로 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_test)
    f1 = f1_score(y_test, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train, y_train)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 07:50:32,721] A new study created in memory with name: no-name-b0343d48-b3f7-49a6-affb-037bbbdc6954
<ipython-input-17-b54e78327f1c>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-17-b54e78327f1c>:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is 

최적의 하이퍼파라미터: {'n_estimators': 390, 'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 13, 'min_samples_leaf': 15, 'min_weight_fraction_leaf': 0.08163046023427645, 'max_features': 'auto', 'max_leaf_nodes': 50, 'min_impurity_decrease': 0.06475786292252089, 'bootstrap': False}
테스트 데이터에서의 F1 점수: 0.0


In [ ]:
# 2. 데이터의 편향된 특성 고려, 데이터를 train, valid, test로 분한 optuna
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import class_weight

# 데이터를 훈련 세트, 검증 세트, 테스트 세트로 나누기
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25, random_state=42)

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 클래스 가중치 계산
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights_dict = dict(enumerate(class_weights))

    # 모델 생성
    model = RandomForestClassifier(**params, class_weight=class_weights_dict)

    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_valid)
    f1 = f1_score(y_valid, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_valid, y_train_valid)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 07:57:54,317] A new study created in memory with name: no-name-9a3cd89c-8dbc-49e1-89d6-e6799ed39668
<ipython-input-19-56a256b6bd96>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-19-56a256b6bd96>:24: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is 

최적의 하이퍼파라미터: {'n_estimators': 73, 'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 12, 'min_samples_leaf': 22, 'min_weight_fraction_leaf': 0.04035168179856026, 'max_features': 'sqrt', 'max_leaf_nodes': 17, 'min_impurity_decrease': 0.01840655222205664, 'bootstrap': True}
테스트 데이터에서의 F1 점수: 0.47058823529411764


In [ ]:
# 3. 결과 이상해서 제출 안함
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 클래스 가중치 계산
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights_dict = dict(enumerate(class_weights))

    # 모델 생성
    model = RandomForestClassifier(**params, class_weight=class_weights_dict)

    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_valid)
    f1 = f1_score(y_valid, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_valid, y_train_valid)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 08:02:18,877] A new study created in memory with name: no-name-db864624-020d-4c32-8559-8eb7a4fc25ea
<ipython-input-20-edb615ecee9d>:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-20-edb615ecee9d>:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
[I 2024-03-30 08:02:21,099] Trial 0 finished with value: 0.5185185185185185 and parameters: {'n_estimators': 669, 'criterion': 'gini', 'max_depth': 37, 'min_samples_split': 24, 'min_samples_leaf': 6, 'min_weight_fraction_leaf': 0.3754158065663793, 'max_features': 

최적의 하이퍼파라미터: {'n_estimators': 14, 'criterion': 'entropy', 'max_depth': 32, 'min_samples_split': 28, 'min_samples_leaf': 15, 'min_weight_fraction_leaf': 0.3328695038507366, 'max_features': 'auto', 'max_leaf_nodes': 104, 'min_impurity_decrease': 0.001813277224726656, 'bootstrap': True}
테스트 데이터에서의 F1 점수: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


# 2. optuna - 오버샘플링
- smote
- ADASYN

In [ ]:
# smote
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE를 사용하여 훈련 세트를 오버샘플링하기
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    model = RandomForestClassifier(**params)

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_test)
    f1 = f1_score(y_test, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_resampled, y_train_resampled)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 11:52:07,874] A new study created in memory with name: no-name-08a846cf-9934-480e-b6fe-bd0690983055
<ipython-input-10-7e51c1ffc077>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-10-7e51c1ffc077>:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is 

최적의 하이퍼파라미터: {'n_estimators': 268, 'criterion': 'entropy', 'max_depth': 70, 'min_samples_split': 25, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0.011912955045092044, 'max_features': 'auto', 'max_leaf_nodes': 66, 'min_impurity_decrease': 0.1622222452805956, 'bootstrap': False}
테스트 데이터에서의 F1 점수: 0.463768115942029


In [ ]:
# ADASYN
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ADASYN를 사용하여 훈련 세트를 오버샘플링하기
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    model = RandomForestClassifier(**params)

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_test)
    f1 = f1_score(y_test, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_resampled, y_train_resampled)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 11:54:42,243] A new study created in memory with name: no-name-3273cade-242d-42fa-8967-4f2ee6a07f89
<ipython-input-11-8673a03eedfb>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-11-8673a03eedfb>:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
[I 2024-03-30 11:54:42,614] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 159, 'criterion': 'entropy', 'max_depth': 59, 'min_samples_split': 5, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0.10834016803726382, 'max_features': 'log2', 'max

최적의 하이퍼파라미터: {'n_estimators': 12, 'criterion': 'entropy', 'max_depth': 46, 'min_samples_split': 3, 'min_samples_leaf': 21, 'min_weight_fraction_leaf': 0.08476915877130797, 'max_features': None, 'max_leaf_nodes': 33, 'min_impurity_decrease': 0.022902484331103548, 'bootstrap': False}
테스트 데이터에서의 F1 점수: 0.5599999999999999


- optuna + under sampling

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

# 데이터를 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 언더샘플링을 사용하여 훈련 세트를 샘플링하기
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    model = RandomForestClassifier(**params)

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_test)
    f1 = f1_score(y_test, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_resampled, y_train_resampled)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 12:21:34,563] A new study created in memory with name: no-name-66394d7b-9a23-487f-9ff2-f1ab76e784f6
<ipython-input-17-a8f557cae267>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-17-a8f557cae267>:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
[I 2024-03-30 12:21:36,141] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 583, 'criterion': 'entropy', 'max_depth': 53, 'min_samples_split': 24, 'min_samples_leaf': 21, 'min_weight_fraction_leaf': 0.0527754423292594, 'max_features': None, 'max_

최적의 하이퍼파라미터: {'n_estimators': 19, 'criterion': 'entropy', 'max_depth': 19, 'min_samples_split': 24, 'min_samples_leaf': 8, 'min_weight_fraction_leaf': 0.029954931270069134, 'max_features': 'sqrt', 'max_leaf_nodes': 93, 'min_impurity_decrease': 0.036521205609975085, 'bootstrap': True}
테스트 데이터에서의 F1 점수: 0.49275362318840576


In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

# 데이터를 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 언더샘플링을 사용하여 훈련 세트를 샘플링하기
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Objective 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 120),
        'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    model = RandomForestClassifier(**params)

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled)

    # 검증 세트에서 F1 점수 계산
    preds_valid = model.predict(X_test)
    f1 = f1_score(y_test, preds_valid)

    return f1

# Optuna study 생성
study = optuna.create_study(direction='maximize')

# Objective 함수를 최적화하여 최적 하이퍼파라미터 찾기
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 하이퍼파라미터를 가진 모델 생성 및 평가
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train_resampled, y_train_resampled)
preds_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, preds_test)
print("테스트 데이터에서의 F1 점수:", test_f1)


[I 2024-03-30 12:25:03,579] A new study created in memory with name: no-name-e51c3b28-249e-4a26-af9f-f7e39cb210f1
<ipython-input-18-a8f557cae267>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_weight_fraction_leaf': trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5),
<ipython-input-18-a8f557cae267>:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_impurity_decrease': trial.suggest_uniform('min_impurity_decrease', 0.0, 0.5),
[I 2024-03-30 12:25:09,477] Trial 0 finished with value: 0.3917525773195876 and parameters: {'n_estimators': 926, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 16, 'min_weight_fraction_leaf': 0.479844457987964, 'max_features':

최적의 하이퍼파라미터: {'n_estimators': 279, 'criterion': 'entropy', 'max_depth': 59, 'min_samples_split': 15, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.19248364721043285, 'max_features': 'sqrt', 'max_leaf_nodes': 66, 'min_impurity_decrease': 0.05356604481411355, 'bootstrap': True}
테스트 데이터에서의 F1 점수: 0.48571428571428565


# 3. 제출

In [ ]:
#제출1. valid data와 test data를 모두 test data로 사용한 경우

df = {'n_estimators':390,
      'criterion' : 'entropy',
      'max_depth' : 100,
      'min_samples_split' : 13,
      'min_samples_leaf' : 15,
      'min_weight_fraction_leaf' : 0.08163046023427645,
      'max_features' : 'auto',
      'max_leaf_nodes' : 50,
      'min_impurity_decrease' : 0.06475786292252089,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0330_1.csv", index=False)

In [ ]:
#제출2. 데이터의 편향된 특성 고려, 데이터를 train, valid, test로 분한 optuna

df = {'n_estimators':73,
      'criterion' : 'entropy',
      'max_depth' : 100,
      'min_samples_split' : 12,
      'min_samples_leaf' : 22,
      'min_weight_fraction_leaf' : 0.04035168179856026,
      'max_features' : 'sqrt',
      'max_leaf_nodes' : 17,
      'min_impurity_decrease' : 0.01840655222205664,
      'bootstrap': True
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0330_2.csv", index=False)

In [ ]:
#제출3 smote-> score 0.5
df = {'n_estimators':268,
      'criterion' : 'entropy',
      'max_depth' : 70,
      'min_samples_split' : 25,
      'min_samples_leaf' : 4,
      'min_weight_fraction_leaf' : 0.011912955045092044,
      'max_features' : 'auto',
      'max_leaf_nodes' : 66,
      'min_impurity_decrease' : 0.1622222452805956,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0330_3.csv", index=False)

In [ ]:
#제출4 ADASYN - score 	0.7325158228
df = {'n_estimators':12,
      'criterion' : 'entropy',
      'max_depth' : 46,
      'min_samples_split' : 3,
      'min_samples_leaf' : 21,
      'min_weight_fraction_leaf' : 0.08476915877130797,
      'max_features' : None,
      'max_leaf_nodes' : 33,
      'min_impurity_decrease' : 0.022902484331103548,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0330_4.csv", index=False)

In [ ]:
#제출5 optuna + balanced, 언더샘플링,  f1-score
df = {'n_estimators':279,
      'criterion' : 'entropy',
      'max_depth' : 59,
      'min_samples_split' : 15,
      'min_samples_leaf' : 5,
      'min_weight_fraction_leaf' : 0.19248364721043285,
      'max_features' : 'sqrt',
      'max_leaf_nodes' : 66,
      'min_impurity_decrease' : 0.05356604481411355,
      'bootstrap': True
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0330_5.csv", index=False)

# 4. 제출 결과

- 제출 1. Optuna - valid data와 test data를 모두 test data로 사용
 - score: 0.7477848101
- 제출 2. Optuna - 데이터의 편향된 특성 고려, 데이터를 train, valid, test로 분리

 - score: 0.7226661392
- 제출 3. smote로 over sampling 후 optuna
 - score: 0.5
- 제출 4. ADASYN로 over sampling 후 optuna
 - score: 0.7325158228
- 제출 5. optuna + 언더샘플링
 - score:0.5

- 예상보다 전체적인 성능이 낮게 나옴
